In [36]:
import cv2
import numpy as np
from scipy import signal

**Helper Method** 
This function takes a frame as input, extracts the red, green, and blue channels of the frame using array slicing, and computes the grayscale value using the formula. It then rounds the grayscale value to the nearest integer and converts it to a uint8 data type using NumPy's astype() function. Finally, it returns the grayscale frame.

In [ ]:
def cvtColor(frame):
    # extract the red, green, and blue channels of the frame
    r = frame[:, :, 0]
    g = frame[:, :, 1]
    b = frame[:, :, 2]

    # compute the grayscale value using the formula
    gray = 0.299 * r + 0.587 * g + 0.114 * b

    # convert the grayscale value to uint8
    gray = np.round(gray).astype(np.uint8)

    return gray

**Helper Method**
This function takes an image as input, along with several optional parameters for computing the derivatives. It first creates Sobel filters for computing the horizontal and vertical derivatives, and then applies the filters to the input image using the scipy.signal.convolve2d() function. It then computes the magnitude and orientation of the derivatives using NumPy's sqrt() and arctan2() functions. Finally, it converts the magnitude to the specified data type using NumPy's astype() function, and returns the derivative in the specified direction (horizontal, vertical, or magnitude). You can adjust the optional parameters to get the desired results.

In [ ]:
def sobel(image, ddepth, dx, dy, ksize=3):
    # create Sobel filters for computing the derivatives
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])

    # compute the horizontal and vertical derivatives using the Sobel filters
    fx = signal.convolve2d(image, sobel_x, mode='same')
    fy = signal.convolve2d(image, sobel_y, mode='same')

    # compute the magnitude and orientation of the derivatives
    mag = np.sqrt(fx**2 + fy**2)
    angle = np.arctan2(fy, fx)

    # convert the magnitude to the specified depth
    if ddepth == -1:
        mag = np.abs(mag)
        mag = np.round(mag * 255 / np.max(mag)).astype(np.uint8)
    else:
        mag = mag.astype(ddepth)

    # return the derivative in the specified direction
    if dx == 1 and dy == 0:
        return fx
    elif dx == 0 and dy == 1:
        return fy
    else:
        return mag

**Main Method**:
This method uses the defined cvtColor() function to convert the frames to grayscale, and the defined Sobel() function to compute the spatial gradients. It then computes the temporal gradient between the two frames by subtracting the grayscale frames, and computes the optical flow vectors using a least-squares solution to the optical flow equation. Finally, it returns the computed flow. You can adjust the optional parameters to get the desired results.

In [ ]:

def calcOpticalFlowFarneback(prev_frame, next_frame, pyr_scale=0.5, levels=3, winsize=15, iterations=3,
                             poly_n=5, poly_sigma=1.2, flags=0):
    # convert frames to grayscale
    prev_gray = cvtColor(prev_frame)
    next_gray = cvtColor(next_frame)
    
    # compute the spatial gradients
    fx = sobel(prev_gray, -1, 1, 0, ksize=3)
    fy = sobel(prev_gray, -1, 0, 1, ksize=3)
    
    # compute the temporal gradient
    ft = next_gray - prev_gray
    
    # compute the optical flow vectors
    flow = np.zeros((prev_gray.shape[0], prev_gray.shape[1], 2))
    for i in range(prev_gray.shape[0]):
        for j in range(prev_gray.shape[1]):
            A = np.array([[np.sum(fx[i,j]**2), np.sum(fx[i,j]*fy[i,j])],
                          [np.sum(fx[i,j]*fy[i,j]), np.sum(fy[i,j]**2)]])
            b = -np.array([np.sum(fx[i,j]*ft[i,j]), np.sum(fy[i,j]*ft[i,j])])
            v = np.linalg.solve(A, b)
            flow[i,j] = v
    
    return flow

References: 
https://theailearner.com/tag/cv2-sobel/

https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html

https://www.geeksforgeeks.org/python-opencv-cv2-cvtcolor-method/

https://docs.opencv.org/3.4/dc/d6b/group__video__track.html

https://www.geeksforgeeks.org/opencv-the-gunnar-farneback-optical-flow/